# Interpolation

PODPAC supports the following interpolation methods with the associated Interpolator classes:
    
```python
INTERPOLATION_METHODS = {
    'optimal': [OptimalInterpolation],
    'nearest_preview': [NearestPreview],
    'nearest': [NearestNeighbor, Rasterio, ScipyGrid, ScipyPoint],
    'bilinear':[Rasterio, ScipyGrid],
    'cubic':[Rasterio],
    'cubic_spline':[Rasterio, ScipyGrid],
    'lanczos':[Rasterio],
    'average':[Rasterio],
    'mode':[Rasterio],
    'gauss':[Rasterio],
    'max':[Rasterio],
    'min':[Rasterio],
    'med':[Rasterio],
    'q1':[Rasterio],
    'q3': [Rasterio],
    'spline_2': [ScipyGrid],
    'spline_3': [ScipyGrid],
    'spline_4': [ScipyGrid],
    'radial': [Radial]
}
```

In [1]:
# notebook imports
from IPython.display import display
import numpy as np
from podpac.coordinates import Coordinates, clinspace
from podpac.data import Array

/Users/marcus/computing/repositories/geospatial/podpac/podpac/datalib/smap.py:203: UserWarning: Could not retrieve SMAP url from PODPAC S3 Server. Using default.HTTPSConnectionPool(host='s3.amazonaws.com', port=443): Max retries exceeded with url: /podpac-s3/settings/nsidc_smap_opendap_url.txt (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1c1fb0aac8>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))
  warnings.warn("Could not retrieve SMAP url from PODPAC S3 Server. Using default." + str(e))


## Nearest Neighbor

In [2]:
# data source 
source = np.array([
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8]
])
native_coords = Coordinates([[0, 5, 10], clinspace('2018-01-01', '2018-01-18', 3)], 
                             dims=['lat', 'time'])
node = Array(source=source, native_coordinates=native_coords)

# coordinates to evaluate node
eval_coords = Coordinates([[.7, 1.2, 9], clinspace('2018-01-01', '2018-01-09', 3)], 
                          dims=['lat', 'time'])

# show node source
node.source

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [3]:
# simple interpolation definition
node.interpolation = 'nearest'
output = node.eval(eval_coords)

# elements are chosen based on their nearest neighbor
output.data

array([[0., 1., 1.],
       [0., 1., 1.],
       [6., 7., 7.]])

In [13]:
# show the interpolation class for this node (generated by definition `node.interpolation = 'nearest'`)
node.interpolation_class

Interpolation
	('default',): nearest, ['NearestNeighbor', 'Rasterio', 'ScipyGrid', 'ScipyPoint'], {}

In [14]:
# show the interpolators that were used to do this interpolation
# this is only available *after* a node has been evaluated
node.interpolators

OrderedDict([(('lat', 'time'),
              <podpac.core.data.interpolate.NearestNeighbor at 0x1c1fb40940>)])

#### Spatial Tolerance

In [6]:
# set a space tolerance on interpolation
node.interpolation = {
    'method': 'nearest',
    'params': {
        'spatial_tolerance': 1.1
    }
}

output = node.eval(eval_coords)

# the 1st element is outside of the space tolerance
output.data

array([[0.41596594, 0.55306712, 0.55306712],
       [       nan,        nan,        nan],
       [0.75520386, 0.6672439 , 0.6672439 ]])

#### Time Tolerance

In [7]:
# set a time tolerance on interpolation
node.interpolation = {
        'method': 'nearest',
        'params': {
            'spatial_tolerance': 1.1,
            'time_tolerance': np.timedelta64(1, 'D')
        }
}

output = node.eval(eval_coords)
output.data

array([[0.41596594,        nan, 0.55306712],
       [       nan,        nan,        nan],
       [0.75520386,        nan, 0.6672439 ]])

## Rasterio Interpolation

Rasterio provides interpolation support for more advanced interpolation methods.
The following interpolation methods are supported by rasterio via podpac:

```python
['nearest', 
 'bilinear', 
 'cubic', 
 'cubic_spline', 
 'lanczos', 
 'average', 
 'mode', 
 'gauss', 
 'max', 
 'min', 
 'med', 
 'q1', 
 'q3']
```

In [15]:
# data source
source = np.array([
    [0, 1, 2, 4, 5],
    [6, 7, 8, 9, 10],
    [11, 12, 13, 14, 15]
])
native_coords = Coordinates([clinspace(0, 10, 3), clinspace(0, 10, 5)], dims=['lat', 'lon'])
node = Array(source=source, native_coordinates=native_coords)

# coordinates to evaluate node
eval_coords = Coordinates([clinspace(1, 11, 3), clinspace(1, 11, 5)], dims=['lat', 'lon'])

# show node data
node.source

array([[ 0,  1,  2,  4,  5],
       [ 6,  7,  8,  9, 10],
       [11, 12, 13, 14, 15]])

In [16]:
# max interpolation from rasterio
node.interpolation = 'max'
output = node.eval(eval_coords)
output.data

array([[ 7.,  8.,  9., 10., 10.],
       [12., 13., 14., 15., 15.],
       [12., 13., 14., 15., 15.]])

In [17]:
# min interpolation
node.interpolation = 'min'
output = node.eval(eval_coords)
output.data

array([[ 0.,  1.,  2.,  4.,  5.],
       [ 6.,  7.,  8.,  9., 10.],
       [11., 12., 13., 14., 15.]])

In [18]:
# bilinear
node.interpolation = 'bilinear'
output = node.eval(eval_coords)
output.data

array([[ 1.6 ,  2.6 ,  3.92,  5.4 ,  6.  ],
       [ 7.4 ,  8.4 ,  9.4 , 10.4 , 11.  ],
       [11.4 , 12.4 , 13.4 , 14.4 , 15.  ]])

In [19]:
# cubic_spline
node.interpolation = 'cubic_spline'
output = node.eval(eval_coords)
output.data

array([[ 2.32254455,  3.2776171 ,  4.56353158,  5.91742631,  6.49712175],
       [ 7.35481887,  8.30344815,  9.33888028, 10.36762404, 10.92533897],
       [10.85637899, 11.80409683, 12.80409683, 13.78684616, 14.34145182]])

## Specify Interpolator (Advanced)

You can specify a single type of interpolator for a data source using the `interpolators` key of the interpolation dictionary definition.

In [21]:
from podpac.interpolators import NearestNeighbor, Rasterio, ScipyGrid, ScipyPoint

In [22]:
# data source
source = np.array([
    [0, 1, 2, 4, 5],
    [6, 7, 8, 9, 10],
    [11, 12, 13, 14, 15]
])
native_coords = Coordinates([clinspace(0, 10, 3), clinspace(0, 10, 5)], dims=['lat', 'lon'])
node = Array(source=source, native_coordinates=native_coords)

# coordinates to evaluate node
eval_coords = Coordinates([clinspace(1, 11, 3), clinspace(1, 11, 5)], dims=['lat', 'lon'])

# show node data
node.source

array([[ 0,  1,  2,  4,  5],
       [ 6,  7,  8,  9, 10],
       [11, 12, 13, 14, 15]])

In [23]:
# use Scipy for nearest neighbor
node.interpolation = {
    'method': 'nearest',
    'interpolators': [ScipyGrid]
}
output = node.eval(eval_coords)
output.data

array([[ 0.,  1.,  2.,  4., nan],
       [ 6.,  7.,  8.,  9., nan],
       [nan, nan, nan, nan, nan]])

## Random Data Source

In [8]:
# random data source 
source = np.random.rand(5, 5)
native_coords = Coordinates([np.linspace(0, 10, 5), np.linspace(0, 10, 5)], 
                             dims=['lat', 'lon'])
node = Array(source=source, native_coordinates=native_coords)

node.source

array([[0.24110384, 0.97631808, 0.18511521, 0.23061487, 0.98810839],
       [0.85195979, 0.02870966, 0.38866469, 0.42147576, 0.70672856],
       [0.03687607, 0.98742923, 0.71115215, 0.49199317, 0.99835041],
       [0.23643553, 0.96888432, 0.17616402, 0.60344384, 0.08679184],
       [0.18609737, 0.53882234, 0.36897557, 0.08814215, 0.91956155]])

In [9]:
# coordinates to evaluate node
eval_coords = Coordinates([[.7, 1.2, 5, 9, 10], [.7, 1.2, 5, 9, 10]], 
                          dims=['lat', 'lon'])